In [2]:
%matplotlib inline
 
import pulp
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from IPython.display import Image


In [ ]:
import pandas as pd

import xml.etree.ElementTree as et

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

with tarfile.open("sample.tar.gz", "r:bz2") as tar:
    xml_path = tar.getnames()[0]
    df = parse_XML(tar.extractfile(xml_path))

In [28]:
data = {'Первая точка':[1,2,2,4],
        'Вторая точка':[2,3,4,3],
        'Поток':[5,7,3,1]
       }

trip_matrix = {'0': [1, 1, 0, 0, 1, 0],
               '1' :[0, 1, 1, 0, 1, 0],
               '2' :[0, 0, 0, 1, 1, 0],
               '3' :[0, 0, 0, 0, 0, 1]}
#1-2 1-3 2-3 2-4 1-4 3-4
 
# Calling DataFrame constructor on list
data = pd.DataFrame(data)
trip_matrix = pd.DataFrame(trip_matrix)
print(data)
print(trip_matrix)

   Первая точка  Вторая точка  Поток
0             1             2      5
1             2             3      7
2             2             4      3
3             4             3      1
   0  1  2  3
0  1  0  0  0
1  1  1  0  0
2  0  1  0  0
3  0  0  1  0
4  1  1  1  0
5  0  0  0  1


In [29]:
prob = pulp.LpProblem('Traffic_matrix', pulp.LpMaximize)

In [30]:
# create flow
flow_variables = []
for trip_num, reb_use in trip_matrix.iterrows():
    variable = str('Pair ' + str(trip_num))
    variable = pulp.LpVariable(str(variable), lowBound = 0, cat = 'Integer')
    flow_variables.append(variable)

In [31]:
sum_flow = 0
for i, flow in enumerate(flow_variables):
        sum_flow += flow
            #вес длина пути + взвешанность?
prob += sum_flow

In [32]:
# Create Constraint 1 - sum X_j
total_flow = []
for rebnum, reb  in data.iterrows():
    total_flow.append(None)
for i, flow in enumerate(flow_variables):
    for trip_num, reb_use in trip_matrix.iterrows():
        if trip_num == i:
            for rebnum, reb  in data.iterrows():
                a = str(rebnum)
                formula = reb_use[a]*flow
                total_flow[rebnum] += formula
        
        
for rebnum, reb  in data.iterrows():           
    prob += (total_flow[rebnum] <= reb['Поток'])

In [33]:
prob.writeLP('Traffic_matrix.lp')
print('🤔 The problem has successfully formulated')

🤔 The problem has successfully formulated


In [34]:
optimization_result = prob.solve()

assert optimization_result == pulp.LpStatusOptimal
print('Status:', pulp.LpStatus[prob.status])
print('Optimal Solution to the problem: ', pulp.value(prob.objective))
print('Individual decision variables: ')


for v in prob.variables():
    if v.varValue >= 0:
        print(v.name, '=', v.varValue)

Status: Optimal
Optimal Solution to the problem:  16.0
Individual decision variables: 
Pair_0 = 5.0
Pair_1 = 0.0
Pair_2 = 7.0
Pair_3 = 3.0
Pair_4 = 0.0
Pair_5 = 1.0
